# [TUTORIAL] Constrained Task Planning (Sweep)

This tutorial is written to instruct basic usage of the task & motion planning pipeline  
One Indy7 robot and several environment geometries will be added and floor-wiping task will be conducted.  
Here, the wiping task is defined as 1) contact with floor, 2) starting waypoint, 3) goal waypoint.  
Thus, any motion that satisfies the constraint will be generated; it may not look like real wiping.  

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, None,
                INDY_IP)]
              , connection_list=[False])

connection_list
[False]


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [4]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"indy0": ((0,0,0), (0,0,-np.pi/2))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## add environment

In [5]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0.001), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=False, fixed=True, collision=False)
floor = gscene.create_safe(GEOTYPE.BOX, "floor_viscol", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (0,0.2,0), 
                           rpy=(np.pi/2,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.2,-0.5,0.001),rpy=(0,0,np.pi/2), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (-0.2,-0.5,0.001), rpy=(0,0,np.pi/2), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
track = gscene.create_safe(GEOTYPE.BOX, "track", "base_link", (0.1,0.5,0.01), (0,-0.5,0), rpy=(0,0,np.pi/2), 
                           color=(1,0.7,0.7,0.5), display=True, fixed=True, collision=False)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (-0.4,-0.3,0.001), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=True, collision=False)

Please create a subscriber to the marker


In [6]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add brush

In [7]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="base_link", dims=(0.2,0.07,0.02), 
                   center=(0,-0.5,0.045), rpy=(0,0,np.pi/2), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="base_link", dims=(0.2,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", dims=(0.19,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")

## add box

In [8]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="obstacle", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.2,-0.5,0.031), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

## init planning scene

In [9]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [11]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
pscene.create_binder(bname="grip0", gname="grip0", rname="indy0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [13]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask, BoxObject

In [14]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
brush = pscene.create_subject(oname="brush", gname="brush_body", _type=CustomObject, 
                             action_points_dict = {"handle": Grasp2Point("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
                                                   "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [15]:
obstacle = pscene.create_subject(oname="obstacle", gname="obstacle", _type=BoxObject, hexahedral=True)

In [16]:
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])})

### planning pipeline

In [17]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

## motion filters

In [18]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
TOOL_LINK = "indy0_tcp"
TOOL_LINK_BUNDLE = ["indy0_tcp", "indy0_link6"]

gcheck = GraspChecker(pscene, end_link_couple_dict= {TOOL_LINK: TOOL_LINK_BUNDLE, "base_link":["base_link"]})
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [19]:
mplan.motion_filters = checkers_all

## ui

In [20]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## Test plan

In [21]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.update_state(crob.home_pose)

In [30]:
gtimer.reset()
for _ in range(10):
    # goal_nodes = [("goal",)]
    goal_nodes = [("goal", "floor", 2)]
    gtimer.tic("plan")
    ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=True)
    gtimer.toc("plan")
    schedules = ppline.tplan.find_schedules()
    schedules_sorted = ppline.sort_schedule(schedules)
    snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

Use 20/20 agents
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = success
node: ('grip0', 'floor', 0)->('floor', 'floor', 0) = fail
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
node: ('grip0', 'floor', 0)->('grip0', 'floor', 1) = fail
branching: 0->1 (0.06/300.0 s, steps/err: 7(27.2891521454 ms)/0.00176621022728)
node: ('grip0', 'floor', 0)->(

node: ('floor', 'grip0', 0)->('floor', 'grip0', 1) = fail
node: ('grip0', 'floor', 0)->('grip0', 'floor', 1) = fail
node: ('goal', 'floor', 0)->('goal', 'grip0', 0) = fail
node: ('goal', 'floor', 0)->('goal', 'floor', 1) = fail
node: ('goal', 'floor', 0)->('goal', 'grip0', 0) = fail
node: ('goal', 'floor', 0)->('grip0', 'floor', 0) = success
node: ('floor', 'goal', 0)->('floor', 'grip0', 0) = fail
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
node: ('floor', 'grip0', 0)->('floor', 'grip0', 1) = fail
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
node: ('goal', 'grip0', 0)->('goal', 'goal', 0) = fail
branching: 2->15 (0.55/300.0 s, steps/err: 34(54.3880462646 ms)/0.00178426220751)
node: ('goal', 'grip0', 0)->('goal', 'grip0', 1) = fail
node: ('floor', 'grip0', 0)->('floor', 'grip0', 1) = fail
node: ('goal', 'grip0', 0)->('goal', 'floor', 0) = fail
node: ('goal', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('goal', 'grip0', 0)->('goal', 'grip0', 1) = fail
branc

node: ('goal', 'grip0', 0)->('goal', 'grip0', 1) = fail
branching: 2->25 (0.77/300.0 s, steps/err: 23(83.389043808 ms)/0.00157807730228)
node: ('floor', 'grip0', 0)->('floor', 'grip0', 1) = fail
node: ('goal', 'grip0', 0)->('goal', 'goal', 0) = fail
node: ('grip0', 'floor', 0)->('grip0', 'floor', 1) = fail
node: ('grip0', 'floor', 0)->('floor', 'floor', 0) = fail
node: ('grip0', 'floor', 0)->('grip0', 'floor', 1) = fail
node: ('grip0', 'floor', 0)->('grip0', 'floor', 1) = fail
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
node: ('goal', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('goal', 'floor', 0)->('grip0', 'floor', 0) = success
node: ('goal', 'floor', 0)->('goal', 'grip0', 0) = fail
node: ('floor', 'goal', 0)->('floor', 'goal', 1) = fail
node: ('floor', 'goal', 0)->('grip0', 'goal', 0) = fail
branching: 8->26 (0.82/300.0 s, steps/err: 49(191.514015198 ms)/0.00103010991816)
node: ('floor', 'floor', 0)->('floo

node: ('goal', 'grip0', 0)->('goal', 'goal', 0) = fail
node: ('goal', 'floor', 0)->('goal', 'grip0', 0) = fail
node: ('grip0', 'floor', 0)->('floor', 'floor', 0) = fail
node: ('grip0', 'floor', 0)->('grip0', 'floor', 1) = fail
node: ('grip0', 'floor', 0)->('grip0', 'floor', 1) = fail
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = success
node: ('floor', 'goal', 0)->('grip0', 'goal', 0) = success
node: ('grip0', 'goal', 0)->('floor', 'goal', 0) = fail
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = success
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
branching: 0->41 (1.07/300.0 s, steps/err: 7(111.332178116 ms)/0.00145894005204)
branching: 17->40 (1.08/300.0 s, steps/err: 34(147.005081177 ms)/0.00172750647166)
node: ('floor', 'goal', 0)->('grip0', 'goal', 0) = fail
branching: 12->38 (1.06/300.0 s, steps/err: 38(82.4830532074 ms)/0.00138371649149)
node: ('floor', 'goal', 0)->('floor', 'goal', 1) = fail
nod

node: ('grip0', 'goal', 1)->('floor', 'goal', 1) = fail
node: ('grip0', 'goal', 0)->('grip0', 'goal', 1) = success
branching: 47->53 (1.28/300.0 s, steps/err: 35(49.8471260071 ms)/0.00152213648649)
node: ('grip0', 'goal', 0)->('grip0', 'goal', 1) = success
branching: 40->54 (1.29/300.0 s, steps/err: 38(56.4720630646 ms)/0.00116708492519)
node: ('grip0', 'floor', 0)->('floor', 'floor', 0) = fail
node: ('grip0', 'goal', 0)->('grip0', 'goal', 1) = success
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = success
node: ('grip0', 'goal', 0)->('grip0', 'goal', 1) = success
branching: 32->56 (1.32/300.0 s, steps/err: 9(31.3229560852 ms)/0.00132185207828)
node: ('grip0', 'floor', 0)->('grip0', 'floor', 1) = fail
node: ('floor', 'goal', 0)->('floor', 'grip0', 0) = fail
branching: 0->55 (1.32/300.0 s, steps/err: 16(46.5710163116 ms)/0.0011836871883)
node: ('grip0', 'goal', 0)->('grip0', 'goal', 1) = success
branching: 32->58 (1.33/300.0 s, steps/err: 8(80.4679393768 ms)/0.00171889645036)
bran

...

node: ('goal', 'floor', 0)->('goal', 'floor', 1) = fail
node: ('floor', 'goal', 1)->('floor', 'goal', 2) = fail
node: ('floor', 'grip0', 1)->('floor', 'floor', 1) = fail
node: ('floor', 'goal', 0)->('grip0', 'goal', 0) = fail
node: ('floor', 'goal', 0)->('floor', 'goal', 1) = fail
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('grip0', 'goal', 0)->('floor', 'goal', 0) = fail
node: ('grip0', 'goal', 1)->('goal', 'goal', 1) = fail
node: ('goal', 'floor', 0)->('goal', 'grip0', 0) = fail
node: ('floor', 'grip0', 0)->('floor', 'goal', 0) = fail
node: ('grip0', 'floor', 0)->('grip0', 'floor', 1) = fail
node: ('grip0', 'floor', 0)->('floor', 'floor', 0) = fail
node: ('floor', 'grip0', 0)->('floor', 'floor', 0) = fail
node: ('goal', 'floor', 0)->('goal', 'grip0', 0) = fail
node: ('grip0', 'floor', 0)->('goal', 'floor', 0) = success
branching: 25->187 (6.49/300.0 s, steps/err: 18(330.275058746 ms)/0.00204396306673)
node: ('floor', 'goal', 1)->('grip0', 'goal', 1) = success
bra

node: ('grip0', 'goal', 1)->('goal', 'goal', 1) = fail
branching: 69->199 (6.98/300.0 s, steps/err: 7(320.466995239 ms)/0.00144886147919)
node: ('grip0', 'floor', 0)->('floor', 'floor', 0) = fail
node: ('floor', 'goal', 0)->('floor', 'goal', 1) = fail
node: ('floor', 'grip0', 0)->('floor', 'goal', 0) = success
branching: 72->200 (7.01/300.0 s, steps/err: 2(801.094055176 ms)/0.00165391381813)
node: ('floor', 'goal', 2)->('grip0', 'goal', 2) = fail
node: ('floor', 'goal', 0)->('grip0', 'goal', 0) = fail
node: ('grip0', 'floor', 0)->('grip0', 'floor', 1) = fail
node: ('grip0', 'goal', 0)->('grip0', 'goal', 1) = fail
node: ('floor', 'goal', 1)->('grip0', 'goal', 1) = fail
node: ('floor', 'goal', 1)->('floor', 'grip0', 1) = fail
node: ('floor', 'grip0', 1)->('floor', 'goal', 1) = fail
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = success
node: ('floor', 'grip0', 0)->('floor', 'goal', 0) = fail
branching: 86->201 (7.1/300.0 s, steps/err: 19(142.988204956 ms)/0.00131609066206)
node: ('

node: ('goal', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('goal', 'grip0', 0)->('goal', 'floor', 0) = fail
node: ('floor', 'grip0', 0)->('floor', 'grip0', 1) = fail
node: ('goal', 'floor', 0)->('goal', 'floor', 1) = fail
node: ('goal', 'floor', 0)->('goal', 'grip0', 0) = fail
node: ('goal', 'grip0', 0)->('goal', 'grip0', 1) = fail
node: ('floor', 'grip0', 1)->('floor', 'floor', 1) = fail
node: ('goal', 'floor', 0)->('goal', 'floor', 1) = fail
node: ('goal', 'grip0', 0)->('goal', 'goal', 0) = fail
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
node: ('floor', 'grip0', 1)->('floor', 'goal', 1) = fail
node: ('floor', 'grip0', 0)->('floor', 'grip0', 1) = fail
node: ('floor', 'goal', 1)->('floor', 'grip0', 1) = fail
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
node: ('floor', 'grip0', 2)->('floor', 'goal', 2) = fail
node: ('floor', 'goal', 1)->('floor', 'grip0', 1) = success
branching: 183->215 (7.56/300.0 s, steps/err: 37(57.2807788849 ms)/0.0012068317069)
nod

node: ('goal', 'grip0', 0)->('goal', 'floor', 0) = fail
node: ('floor', 'grip0', 1)->('floor', 'floor', 1) = fail
node: ('goal', 'grip0', 0)->('goal', 'grip0', 1) = fail
node: ('floor', 'goal', 2)->('grip0', 'goal', 2) = fail
node: ('goal', 'floor', 0)->('goal', 'grip0', 0) = fail
node: ('floor', 'goal', 1)->('floor', 'goal', 2) = fail
node: ('grip0', 'goal', 1)->('goal', 'goal', 1) = fail
node: ('floor', 'grip0', 1)->('floor', 'floor', 1) = fail
node: ('floor', 'goal', 2)->('grip0', 'goal', 2) = fail
node: ('goal', 'grip0', 0)->('goal', 'grip0', 1) = fail
node: ('grip0', 'floor', 0)->('floor', 'floor', 0) = fail
node: ('floor', 'grip0', 0)->('floor', 'grip0', 1) = fail
node: ('floor', 'goal', 0)->('grip0', 'goal', 0) = fail
node: ('goal', 'floor', 0)->('goal', 'floor', 1) = fail
node: ('goal', 'floor', 0)->('goal', 'floor', 1) = fail
node: ('floor', 'grip0', 2)->('floor', 'goal', 2) = fail
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
node: ('floor', 'goal', 0)->('grip0', 

node: ('goal', 'grip0', 0)->('goal', 'grip0', 1) = fail
node: ('floor', 'goal', 0)->('grip0', 'goal', 0) = fail
node: ('grip0', 'goal', 0)->('floor', 'goal', 0) = fail
======================= terminated 7: first answer acquired ===============================
branching: 234->242 (8.56/300.0 s, steps/err: 28(94.3682193756 ms)/0.0014439240483)
branching: 37->243 (8.56/300.0 s, steps/err: 8(655.246973038 ms)/0.00184810760045)
======================= terminated 4: first answer acquired from other agent ===============================
======================= terminated 13: first answer acquired from other agent ===============================
======================= terminated 10: first answer acquired ===============================
======================= terminated 19: first answer acquired from other agent ===============================
======================= terminated 1: first answer acquired from other agent ===============================
node: ('floor', 'goal', 2)->('grip0', 'goa

In [31]:
print(gtimer)

plan: 	74727.0 ms/10 = 7472.695 ms (3465.893/13270.64)



## play searched plan

In [32]:
ppline.play_schedule(snode_schedule[:2], period=0.01)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [33]:
ppline.play_schedule(snode_schedule, period=0.03)

In [36]:
tuple(binding for binding in [1,2,3])

(1, 2, 3)